# 抽出型文書要約

## 手続き

1. 前準備
1. クリーニング(文区切り，絵文字/記号などの削除)
1. 文章の正規化(全角半角変換，数字の削除)
1. 文章の単語分割(形態素解析)
1. 単語の正規化(同じ意味を持つ単語の統一)
1. ストップワードの除去(名詞/形容詞/副詞/動詞のみの利用，辞書による除去)
1. 文書のベクトル表現(TF-IDF)
1. 要約モデルの適用(LexRank)
1. FlaskでAPIに

## 実装

### 1. 前準備

In [650]:
%load_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd

import neologdn

# 文区切り
import functools
from ja_sentence_segmenter.common.pipeline import make_pipeline
from ja_sentence_segmenter.concatenate.simple_concatenator import concatenate_matching
from ja_sentence_segmenter.normalize.neologd_normalizer import normalize
from ja_sentence_segmenter.split.simple_splitter import split_newline, split_punctuation

# クリーニング
import re
from modules import data_cleaning_ks as dc

# MeCab(形態素解析)
import MeCab as mecab
mecab = mecab.Tagger('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# 単語の正規化
#import emoji
#import mojimoji

# ストップワード除去
# 名詞/形容詞/副詞/動詞のみを抽出できるライブラリのインポート
import mecabpr
mpr = mecabpr.MeCabPosRegex('-r /etc/mecabrc -d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd')

# キーフレーズ抽出モデル(YAKE!)
import yake
from yake import KeywordExtractor

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### 2. クリーニング

In [839]:
split_punc2 = functools.partial(split_punctuation, punctuations=r"。.!！?？")
concat_tail_te = functools.partial(concatenate_matching, former_matching_rule=r"^(?P<result>.+)([\r\n]+)$", remove_former_matched=False)
segmenter = make_pipeline(normalize, split_newline, concat_tail_te, split_punc2)

text_org = """
最近キーフレーズ抽出に興味がありいろいろと調べていたら、BERT版のキーフレーズ抽出パッケージがあったので試してみた。ちなみにキーフレーズ抽出とは、「文章からその主題を良く表現している句を抽出する技術」のこと。
環境は、Docker for Mac を想定。
DecktopにあるPreference -> Advance でMemoryの上限を調整する。KeyBERTをインストールするとPytorchをインストールしようとするが、Dockerのデフォルト設定のままだとメモリ不足でインストールに失敗する。なので、設定を変更してメモリ制限を解除する。今回は12Gに設定した。"""

In [840]:
# 『楽しみ〜〜〜』の『〜〜〜』などを削除
text = neologdn.normalize(text_org)

In [841]:
sentences = list(segmenter(text))
pd.DataFrame(sentences)

,0
0,最近キーフレーズ抽出に興味がありいろいろと調べていたら、BERT版のキーフレーズ抽出パッケー...
1,ちなみにキーフレーズ抽出とは、「文章からその主題を良く表現している句を抽出する技術」のこと。
2,環境は、Docker for Macを想定。
3,DecktopにあるPreference->AdvanceでMemoryの上限を調整する。
4,KeyBERTをインストールするとPytorchをインストールしようとするが、Dockerの...
5,なので、設定を変更してメモリ制限を解除する。
6,今回は12Gに設定した。


### 3. 文章の正規化

In [842]:
sentences_dc = dc.data_cleaning(sentences)
pd.DataFrame(sentences_dc)

,0
0,最近キーフレーズ抽出に興味がありいろいろと調べていたらbert版のキーフレーズ抽出パッケージ...
1,ちなみにキーフレーズ抽出とは文章からその主題を良く表現している句を抽出する技術のこと
2,環境はdockerformacを想定
3,decktopにあるpreferenceadvanceでmemoryの上限を調整する
4,keybertをインストールするとpytorchをインストールしようとするがdockerのデ...
5,なので設定を変更してメモリ制限を解除する
6,今回は12gに設定した


In [843]:
sentence_words = []
result = []
for i, sentence in enumerate(sentences_dc):
    # 名詞|形容詞|副詞|動詞のみの抽出
    ma = sum(mpr.findall(sentence, "(名詞-固有名詞|名詞-一般|名詞-サ変接続|名詞-形容動詞語幹)", raw=True), [])
    print(ma)
    # 活用形の統一(基本形へ)
    sentence_ma = []
    for word in ma:
        if word.split(',')[6] == "*":
            sentence_ma.append(word.split('\t')[0])
        else:
            sentence_ma.append(word.split(',')[6])
    result.append(sentence_ma)

['キーフレーズ\t名詞,固有名詞,一般,*,*,*,キーフレーズ,キーフレーズ,キーフレーズ', '抽出\t名詞,サ変接続,*,*,*,*,抽出,チュウシュツ,チューシュツ', '興味\t名詞,一般,*,*,*,*,興味,キョウミ,キョーミ', 'bert\t名詞,一般,*,*,*,*,*', 'キーフレーズ\t名詞,固有名詞,一般,*,*,*,キーフレーズ,キーフレーズ,キーフレーズ', '抽出\t名詞,サ変接続,*,*,*,*,抽出,チュウシュツ,チューシュツ', 'パッケージ\t名詞,一般,*,*,*,*,パッケージ,パッケージ,パッケージ']
['キーフレーズ\t名詞,固有名詞,一般,*,*,*,キーフレーズ,キーフレーズ,キーフレーズ', '抽出\t名詞,サ変接続,*,*,*,*,抽出,チュウシュツ,チューシュツ', '文章\t名詞,一般,*,*,*,*,文章,ブンショウ,ブンショー', '主題\t名詞,一般,*,*,*,*,主題,シュダイ,シュダイ', '表現\t名詞,サ変接続,*,*,*,*,表現,ヒョウゲン,ヒョーゲン', '句\t名詞,一般,*,*,*,*,句,ク,ク', '抽出\t名詞,サ変接続,*,*,*,*,抽出,チュウシュツ,チューシュツ', '技術\t名詞,一般,*,*,*,*,技術,ギジュツ,ギジュツ']
['環境\t名詞,一般,*,*,*,*,環境,カンキョウ,カンキョー', 'dockerformac\t名詞,一般,*,*,*,*,*', '想定\t名詞,サ変接続,*,*,*,*,想定,ソウテイ,ソーテイ']
['decktop\t名詞,一般,*,*,*,*,*', 'preferenceadvance\t名詞,一般,*,*,*,*,*', 'memory\t名詞,一般,*,*,*,*,*', '上限\t名詞,一般,*,*,*,*,上限,ジョウゲン,ジョーゲン', '調整\t名詞,サ変接続,*,*,*,*,調整,チョウセイ,チョーセイ']
['keybert\t名詞,一般,*,*,*,*,*', 'インストール\t名詞,一般,*,*,*,*,インストール,インストール,インストール', 'pytorch\t名詞,一般,*,*,*,*,*', 'インストール\t名詞,一般,*,*,*,*,インストール,インストール,インストール',

In [844]:
result

[['キーフレーズ', '抽出', '興味', 'bert', 'キーフレーズ', '抽出', 'パッケージ'],
 ['キーフレーズ', '抽出', '文章', '主題', '表現', '句', '抽出', '技術'],
 ['環境', 'dockerformac', '想定'],
 ['decktop', 'preferenceadvance', 'memory', '上限', '調整'],
 ['keybert',
  'インストール',
  'pytorch',
  'インストール',
  'しよう',
  'docker',
  'デフォルト',
  '設定',
  'メモリ',
  '不足',
  'インストール',
  '失敗'],
 ['設定', '変更', 'メモリ', '制限', '解除'],
 ['12g', '設定']]

In [845]:
for i, sentence in enumerate(result):
    for j, word in  enumerate(sentence):
        if "ー" in word:
            result[i][j] = word.replace("ー","-")
        if re.search(r'\d', word):
            result[i][j] = ""

In [846]:
result

[['キ-フレ-ズ', '抽出', '興味', 'bert', 'キ-フレ-ズ', '抽出', 'パッケ-ジ'],
 ['キ-フレ-ズ', '抽出', '文章', '主題', '表現', '句', '抽出', '技術'],
 ['環境', 'dockerformac', '想定'],
 ['decktop', 'preferenceadvance', 'memory', '上限', '調整'],
 ['keybert',
  'インスト-ル',
  'pytorch',
  'インスト-ル',
  'しよう',
  'docker',
  'デフォルト',
  '設定',
  'メモリ',
  '不足',
  'インスト-ル',
  '失敗'],
 ['設定', '変更', 'メモリ', '制限', '解除'],
 ['', '設定']]

In [847]:
corpus = [' '.join(sentence) + u'. ' for sentence in result]
"".join(corpus)

'キ-フレ-ズ 抽出 興味 bert キ-フレ-ズ 抽出 パッケ-ジ. キ-フレ-ズ 抽出 文章 主題 表現 句 抽出 技術. 環境 dockerformac 想定. decktop preferenceadvance memory 上限 調整. keybert インスト-ル pytorch インスト-ル しよう docker デフォルト 設定 メモリ 不足 インスト-ル 失敗. 設定 変更 メモリ 制限 解除.  設定. '

In [848]:
corpus

['キ-フレ-ズ 抽出 興味 bert キ-フレ-ズ 抽出 パッケ-ジ. ',
 'キ-フレ-ズ 抽出 文章 主題 表現 句 抽出 技術. ',
 '環境 dockerformac 想定. ',
 'decktop preferenceadvance memory 上限 調整. ',
 'keybert インスト-ル pytorch インスト-ル しよう docker デフォルト 設定 メモリ 不足 インスト-ル 失敗. ',
 '設定 変更 メモリ 制限 解除. ',
 ' 設定. ']

### 7.キーフレーズ抽出(YAKE!)の適用

In [849]:
language = "ja"
max_ngram_size = 1
deduplication_thresold = 0.5
numOfKeywords = 10

custom_kw_extractor = yake.KeywordExtractor(lan=language, n=max_ngram_size, dedupLim=deduplication_thresold, top=numOfKeywords, features=None)
keywords = custom_kw_extractor.extract_keywords("".join(corpus))
keywords = np.array(keywords)[:,0]
keywords

array(['bert', 'パッケ-ジ', 'インスト-ル', 'メモリ', 'decktop', 'dockerformac',
       'preferenceadvance', 'memory', 'pytorch', 'しよう'], dtype='<U19')

In [850]:
for i, word in enumerate(keywords):
    if "-" in word:
        keywords[i] = word.replace("-","ー")

In [851]:
keywords

array(['bert', 'パッケージ', 'インストール', 'メモリ', 'decktop', 'dockerformac',
       'preferenceadvance', 'memory', 'pytorch', 'しよう'], dtype='<U19')

### 8. モジュールの動作確認

In [714]:
%load_ext autoreload
%autoreload 2
from modules import extractive_summarization as es

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ImportError: cannot import name 'extractive_summarization'

In [637]:
text = """
長岡は美味しいラーメン屋がたくさんある．
その中でも特に美味しい(と個人的に思う)ラーメン屋を紹介したいと思う．
それは…「おこじょ」である．
ここは，限定ラーメンを月1ペースで出しており，何度行っても新しい味が楽しめるラーメン屋である．
現在(2021/9)は，うなぎハモラーメンを提供している．
これは，スープが泡立っているため，大変軽い舌触りであり，味もクリーミーに仕上がっている．
トッピングのハモは，ナスと一緒に食べることが推奨されており，ふわふわかつジューシーな味わいが楽しめる．
お値段は1,500円と少し高めだが，値段分の価値は十分だと考えている．
皆さんも機会があればぜひ食べてもらいたい．
"""

In [36]:
summary = es.preprocessed_lexrank(text, sum_count=3)

In [37]:
print(summary)

長岡は美味しいラーメン屋がたくさんある。
それは…「おこじょ」である。
ここは,限定ラーメンを月1ペースで出しており,何度行っても新しい味が楽しめるラーメン屋である。

